In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory

2024-09-24 12:14:15.273209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 12:14:15.351446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 12:14:15.364235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 12:14:15.463702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-24 12:14:16.690343: W tensorflow/compiler/tf2

In [6]:
# Load dataset from directory
train_dir = 'datasets/train'
test_dir = 'datasets/test'

# Load train and test datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int', 
    shuffle=True,
    seed=123 
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=False, 
    seed=123
)

Found 1034 files belonging to 3 classes.


I0000 00:00:1727176459.136012   20028 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727176459.292481   20028 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727176459.297268   20028 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727176459.302524   20028 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 128 files belonging to 3 classes.


In [7]:
# Function to extract features from the images using CNN
def extract_features(dataset, cnn_model):
    features = []
    labels = []

    for images, lbls in dataset:
        try:
            feature_batch = cnn_model.predict(images)
            features.append(feature_batch)
            labels.append(lbls.numpy()) # Convert labels to numpy arrays
        except Exception as e:
            print("Error processing a batch:", e)
            continue

    features = np.vstack(features) # Shape to (samples, features)
    labels = np.concatenate(labels) # Combine all label batches
    return features, labels

In [8]:
# Load pre-trained CNN model
cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [9]:
# Extract features from train and test sets
train_features, train_labels = extract_features(train_dataset, cnn_model)
test_features, test_labels = extract_features(test_dataset, cnn_model)

2024-09-24 12:14:48.679648: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
I0000 00:00:1727176490.282642   24189 service.cc:146] XLA service 0x7fef34003550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727176490.282664   24189 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-09-24 12:14:50.371775: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-24 12:14:50.872723: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


I0000 00:00:1727176493.917920   24189 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-09-24 12:14:54.062107: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


2024-09-24 12:14:54.204635: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-09-24 12:14:54.346363: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


...

2024-09-24 12:14:54.481816: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━

2024-09-24 12:15:02.383662: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


2024-09-24 12:15:03.479566: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
# Flatten the features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [8]:
# KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(train_features, train_labels)

# Predict and evaluate
test_predictions = knn_model.predict(test_features)
train_preditions = knn_model.predict(train_features)
test_accuracy = accuracy_score(test_labels, test_predictions)
train_accuracy = accuracy_score(train_labels, train_preditions)
print(f"Test Accuracy: {test_accuracy:.3f}")
print("Train accuracy: {:.3f}".format(train_accuracy))

Test Accuracy: 0.703
Train accuracy: 0.795


In [9]:
import pickle

with open("models/resnet50_knn_ensemble.pkl", "wb") as file:
    pickle.dump(knn_model, file)

In [3]:
import pickle
from sklearn.metrics import accuracy_score, classification_report # type: ignore

In [4]:
with open("models/resnet50_knn_ensemble.pkl", 'rb') as file:
    model = pickle.load(file)

In [12]:
class_names = ["angular_leaf_spot", "bean_rust", "healthy"]

In [11]:
test_pred = model.predict(test_features)

In [13]:
report = classification_report(test_labels, test_pred, target_names=class_names)

In [15]:
print(report)

                   precision    recall  f1-score   support

angular_leaf_spot       0.62      0.86      0.72        43
        bean_rust       0.76      0.30      0.43        43
          healthy       0.78      0.95      0.86        42

         accuracy                           0.70       128
        macro avg       0.72      0.71      0.67       128
     weighted avg       0.72      0.70      0.67       128

